## Install all the requirements

In [ ]:
!pip install -U transformers 
!pip install huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 3.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.3.0 requires tokenizers<0.20.0,>=0.19.1, but you have tokenizers 0.20.2 which is incompatible.
autorag 0.3.8 requires numpy<2.0.0, but you have numpy 2.1.2 which is incompatible.


## Log in to Hugging Face using your Token
If you can't paste it here, run these commands in the terminal.

In [ ]:
!git config --global credential.helper store
!huggingface-cli login

In [ ]:
import torch 
from PIL import Image

from transformers import (
    MllamaForConditionalGeneration,
    AutoProcessor,
    GenerationConfig,
)

## Loading the model

🚨 REMINDER: You need access to the model to use it.
Check the authorization [here](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision)

In [ ]:
model_id = "meta-llama/Llama-3.2-11B-Vision"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.gradient_checkpointing_enable()

processor = AutoProcessor.from_pretrained(model_id)

## Function definition

In [ ]:
def answer_image_question(prompt, image_file):
  image = Image.open(image_file)
  prompt = f"<|image|><|begin_of_text|>{prompt}"

  inputs = processor(image, prompt, return_tensors="pt").to("cpu")
  generation_config = GenerationConfig.from_pretrained(model_id)
  generation_config.gradient_checkpointing = True
  output = model.generate(**inputs, generation_config=generation_config, 
                          max_new_tokens=250)

  return processor.decode(output[0])

## Testing the function

The time execution will depend on your machine components. (RAM, GPU, etc)

In [ ]:
# === CALLING THE FUNCTION ===
image_file = "./imgs/ai-generated-stray-cat-in-danger-background-animal-background-photo.jpg"
prompt = "What do you see in this image?"
answer = answer_image_question(prompt, image_file)
print(answer)